# Classroom Scheduling Optimization

A comprehensive school timetabling example using constraint programming.

## Problem Overview

Assign each **lesson** to a **timeslot** and a **room** while optimizing for:

**Hard Constraints (Must Be Satisfied):**
- Room conflict: No double-booking rooms
- Teacher conflict: Teachers can only teach one lesson at a time
- Student conflict: Student groups attend one lesson at a time
- Room capacity: Rooms must fit all students
- Room capabilities: Rooms must have required equipment (chemistry needs lab, etc.)
- Travel time: Students must be able to reach their next class

**Soft Constraints (Preferences):**
- Minimize walking distance between consecutive lessons
- Teacher time preferences (morning/afternoon)
- Avoid gaps in student schedules
- Spread lessons across the week
- Schedule difficult subjects in the morning
- Wheelchair accessibility prioritization

**Optimization Goals:**
- Maximize teacher satisfaction
- Maximize schedule compactness


## Setup

Load the solver library.

**Build first:** `cd kotlin && ./gradlew :solver:shadowJar`


In [1]:
// Load the fat JAR with all dependencies bundled
@file:DependsOn("../kotlin/solver/build/libs/solver-all-0.1.0-SNAPSHOT.jar")
@file:DependsOn("com.fasterxml.jackson.core:jackson-databind:2.17.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.0")
@file:DependsOn("com.fasterxml.jackson.dataformat:jackson-dataformat-yaml:2.17.0")

import arrow.core.*
import io.github.andreifilonenko.kpsat.dsl.*
import io.github.andreifilonenko.kpsat.solver.*
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.databind.DeserializationFeature
import com.fasterxml.jackson.dataformat.yaml.YAMLFactory
import com.fasterxml.jackson.module.kotlin.registerKotlinModule
import java.io.File
import java.time.DayOfWeek
import java.time.LocalTime
import kotlin.math.abs

// Load OR-Tools native library
com.google.ortools.Loader.loadNativeLibraries()

println("Libraries loaded: DSL, Solver, Arrow, OR-Tools, Jackson YAML")


Libraries loaded: DSL, Solver, Arrow, OR-Tools, Jackson YAML


## Domain Model

Define the core entities with room capabilities, subject requirements, and distance calculations.


In [2]:
// ═══════════════════════════════════════════════════════════════════════════
// ROOM CAPABILITIES - Equipment and facilities available in rooms
// ═══════════════════════════════════════════════════════════════════════════

enum class RoomCapability(val code: String, val description: String) {
    PROJECTOR("PROJ", "Digital projector"),
    WHITEBOARD("WB", "Standard whiteboard"),
    SMARTBOARD("SB", "Interactive smartboard"),
    CHEMICAL_LAB("CHEM", "Chemistry laboratory equipment"),
    PHYSICS_LAB("PHYS", "Physics laboratory equipment"),
    BIOLOGY_LAB("BIO", "Biology laboratory with microscopes"),
    COMPUTER_STATIONS("COMP", "Computer workstations"),
    AUDIO_SYSTEM("AUD", "Professional audio system"),
    ART_SUPPLIES("ART", "Art supplies and easels"),
    MUSIC_EQUIPMENT("MUS", "Musical instruments"),
    GYM_EQUIPMENT("GYM", "Gymnasium equipment"),
    OUTDOOR_ACCESS("OUT", "Direct outdoor access"),
    WHEELCHAIR_ACCESSIBLE("ACC", "Full wheelchair accessibility")
}

// ═══════════════════════════════════════════════════════════════════════════
// TIMESLOT - A period during which lessons can be scheduled
// ═══════════════════════════════════════════════════════════════════════════

data class Timeslot(
    val id: Int,
    val dayOfWeek: DayOfWeek,
    val startTime: LocalTime,
    val endTime: LocalTime,
    val periodNumber: Int  // 1-6 within the day
) {
    val isAfternoon: Boolean get() = startTime.hour >= 12
    val isMorning: Boolean get() = startTime.hour < 12
    val isFirstPeriod: Boolean get() = periodNumber == 1
    val isLastPeriod: Boolean get() = periodNumber == 6
    
    override fun toString() = "${dayOfWeek.toString().take(3)} P$periodNumber (${startTime}-${endTime})"
}

// ═══════════════════════════════════════════════════════════════════════════
// ROOM - A classroom with location, capacity, and capabilities
// ═══════════════════════════════════════════════════════════════════════════

data class Room(
    val id: Int,
    val name: String,
    val building: String,
    val floor: Int,
    val capacity: Int,
    val capabilities: Set<RoomCapability>
) {
    fun hasCapability(cap: RoomCapability) = cap in capabilities
    fun hasAllCapabilities(caps: Set<RoomCapability>) = capabilities.containsAll(caps)
    fun hasAnyCapability(caps: Set<RoomCapability>) = caps.isEmpty() || caps.any { it in capabilities }
    
    val capabilityCodes: String get() = capabilities.joinToString(" ") { it.code }
}

// ═══════════════════════════════════════════════════════════════════════════
// SUBJECT - A course with equipment requirements
// ═══════════════════════════════════════════════════════════════════════════

enum class TimePreference { MORNING, AFTERNOON, ANY }

data class Subject(
    val id: String,  // Uses shortName as id for readability
    val name: String,
    val requiredCapabilities: Set<RoomCapability>,  // Must have ALL of these
    val preferredCapabilities: Set<RoomCapability>, // Nice to have
    val difficulty: Int,  // 1-5 scale
    val preferredTime: TimePreference = TimePreference.ANY,
    val color: String  // For visualization
) {
    val shortName: String get() = id  // id IS the shortName
}

// ═══════════════════════════════════════════════════════════════════════════
// TEACHER - An instructor with preferences
// ═══════════════════════════════════════════════════════════════════════════

data class Teacher(
    val id: Int,
    val name: String,
    val subjects: Set<String>,  // Subject IDs (shortNames) they can teach
    val prefersMorning: Boolean = true,
    val maxLessonsPerDay: Int = 6,
    val unavailableDays: Set<DayOfWeek> = emptySet(),
    val needsAccessibleRoom: Boolean = false
)

// ═══════════════════════════════════════════════════════════════════════════
// STUDENT GROUP - A class of students
// ═══════════════════════════════════════════════════════════════════════════

data class StudentGroup(
    val id: Int,
    val name: String,
    val size: Int,
    val grade: Int,
    val hasWheelchairStudent: Boolean = false
)

// ═══════════════════════════════════════════════════════════════════════════
// LESSON - A specific class session to be scheduled
// ═══════════════════════════════════════════════════════════════════════════

data class Lesson(
    val id: Int,
    val subject: Subject,
    val teacher: Teacher,
    val studentGroup: StudentGroup
)

println("Domain model defined with ${RoomCapability.values().size} room capabilities")


Domain model defined with 13 room capabilities


In [3]:
// ═══════════════════════════════════════════════════════════════════════════
// YAML DATA CLASSES - Type-safe parsing of school-data.yaml
// ═══════════════════════════════════════════════════════════════════════════

data class PeriodYaml(
    val number: Int = 0,
    val start: String = "",
    val end: String = ""
)

data class TimeslotConfigYaml(
    val days: List<String> = emptyList(),
    val periods: List<PeriodYaml> = emptyList()
)

data class RoomYaml(
    val id: Int = 0,
    val name: String = "",
    val building: String = "",
    val floor: Int = 0,
    val capacity: Int = 0,
    val capabilities: List<String> = emptyList()
)

data class SubjectYaml(
    val id: String = "",
    val name: String = "",
    val requiredCapabilities: List<String> = emptyList(),
    val preferredCapabilities: List<String> = emptyList(),
    val difficulty: Int = 1,
    val preferredTime: String = "ANY",
    val color: String = ""
)

data class TeacherYaml(
    val id: Int = 0,
    val name: String = "",
    val subjects: List<String> = emptyList(),
    val prefersMorning: Boolean = true,
    val maxLessonsPerDay: Int = 6,
    val unavailableDays: List<String> = emptyList(),
    val needsAccessibleRoom: Boolean = false
)

data class StudentGroupYaml(
    val id: Int = 0,
    val name: String = "",
    val size: Int = 0,
    val grade: Int = 0,
    val hasWheelchairStudent: Boolean = false
)

data class BuildingDistanceYaml(
    val buildings: List<String> = emptyList(),
    val baseMinutes: Int = 0
)

data class DistanceConfigYaml(
    val sameBuildingPerFloor: Int = 1,
    val buildingDistances: List<BuildingDistanceYaml> = emptyList(),
    val defaultBetweenBuildings: Int = 5
)

data class SchoolDataYaml(
    val timeslotConfig: TimeslotConfigYaml = TimeslotConfigYaml(),
    val rooms: List<RoomYaml> = emptyList(),
    val subjects: List<SubjectYaml> = emptyList(),
    val teachers: List<TeacherYaml> = emptyList(),
    val studentGroups: List<StudentGroupYaml> = emptyList(),
    val curriculum: Map<String, Int> = emptyMap(),
    val distanceConfig: DistanceConfigYaml = DistanceConfigYaml()
)

println("YAML data classes defined")


YAML data classes defined


## Load Data from YAML

Load school data from `school-data.yaml` - rooms, subjects, teachers, student groups, and curriculum.


In [4]:
// ═══════════════════════════════════════════════════════════════════════════
// LOAD YAML DATA WITH JACKSON
// ═══════════════════════════════════════════════════════════════════════════

val mapper = ObjectMapper(YAMLFactory())
    .registerKotlinModule()
    .configure(DeserializationFeature.FAIL_ON_UNKNOWN_PROPERTIES, false)

val schoolData = mapper.readValue(File("school-data.yaml"), SchoolDataYaml::class.java)

println("Loaded school-data.yaml with Jackson")

// ═══════════════════════════════════════════════════════════════════════════
// PARSE TIMESLOTS
// ═══════════════════════════════════════════════════════════════════════════

val configDays = schoolData.timeslotConfig.days.map { DayOfWeek.valueOf(it) }
val configPeriods = schoolData.timeslotConfig.periods

val timeslots = configDays.flatMapIndexed { dayIdx, day ->
    configPeriods.mapIndexed { periodIdx, period ->
        val startParts = period.start.split(":")
        val endParts = period.end.split(":")
        Timeslot(
            id = dayIdx * configPeriods.size + periodIdx,
            dayOfWeek = day,
            startTime = LocalTime.of(startParts[0].toInt(), startParts[1].toInt()),
            endTime = LocalTime.of(endParts[0].toInt(), endParts[1].toInt()),
            periodNumber = period.number
        )
    }
}

println("Parsed ${timeslots.size} timeslots (${configDays.size} days x ${configPeriods.size} periods)")


Loaded school-data.yaml with Jackson
Parsed 30 timeslots (5 days x 6 periods)


In [5]:
// ═══════════════════════════════════════════════════════════════════════════
// PARSE ROOMS
// ═══════════════════════════════════════════════════════════════════════════

val rooms = schoolData.rooms.map { r ->
    Room(
        id = r.id,
        name = r.name,
        building = r.building,
        floor = r.floor,
        capacity = r.capacity,
        capabilities = r.capabilities.map { RoomCapability.valueOf(it) }.toSet()
    )
}

// Build distance matrix using config
val distanceConfig = schoolData.distanceConfig
val sameBuildingPerFloor = distanceConfig.sameBuildingPerFloor
val buildingDistances = distanceConfig.buildingDistances.associate { bd ->
    bd.buildings.toSet() to bd.baseMinutes
}
val defaultBetweenBuildings = distanceConfig.defaultBetweenBuildings

fun walkingTime(room1: Room, room2: Room): Int {
    if (room1.id == room2.id) return 0
    return if (room1.building == room2.building) {
        maxOf(1, abs(room1.floor - room2.floor) * sameBuildingPerFloor)
    } else {
        val baseTime = buildingDistances[setOf(room1.building, room2.building)] ?: defaultBetweenBuildings
        baseTime + abs(room1.floor - room2.floor)
    }
}

val distanceMatrix = rooms.flatMap { r1 ->
    rooms.map { r2 -> (r1.id to r2.id) to walkingTime(r1, r2) }
}.toMap()

println("Parsed ${rooms.size} rooms across ${rooms.map { it.building }.distinct().size} buildings")
println("  Buildings: ${rooms.groupBy { it.building }.map { "${it.key} (${it.value.size})" }.joinToString(", ")}")


Parsed 14 rooms across 4 buildings
  Buildings: Main (5), Science (4), Arts (2), Sports (3)


In [6]:
// ═══════════════════════════════════════════════════════════════════════════
// PARSE SUBJECTS
// ═══════════════════════════════════════════════════════════════════════════

val subjects = schoolData.subjects.map { s ->
    Subject(
        id = s.id,
        name = s.name,
        requiredCapabilities = s.requiredCapabilities.map { RoomCapability.valueOf(it) }.toSet(),
        preferredCapabilities = s.preferredCapabilities.map { RoomCapability.valueOf(it) }.toSet(),
        difficulty = s.difficulty,
        preferredTime = TimePreference.valueOf(s.preferredTime),
        color = s.color
    )
}

val subjectById = subjects.associateBy { it.id }

println("Parsed ${subjects.size} subjects")
subjects.filter { it.requiredCapabilities.isNotEmpty() }.forEach { 
    println("  ${it.name} requires: ${it.requiredCapabilities.joinToString { c -> c.code }}")
}


Parsed 10 subjects
  Chemistry requires: CHEM
  Physics requires: PHYS
  Biology requires: BIO
  Computer Science requires: COMP
  Art requires: ART
  Music requires: MUS


In [7]:
// ═══════════════════════════════════════════════════════════════════════════
// PARSE TEACHERS
// ═══════════════════════════════════════════════════════════════════════════

val teachers = schoolData.teachers.map { t ->
    Teacher(
        id = t.id,
        name = t.name,
        subjects = t.subjects.toSet(),
        prefersMorning = t.prefersMorning,
        maxLessonsPerDay = t.maxLessonsPerDay,
        unavailableDays = t.unavailableDays.map { DayOfWeek.valueOf(it) }.toSet(),
        needsAccessibleRoom = t.needsAccessibleRoom
    )
}

val teacherById = teachers.associateBy { it.id }

println("Parsed ${teachers.size} teachers")
teachers.forEach { t ->
    val subjectNames = t.subjects.map { subjectById[it]?.shortName ?: "?" }.joinToString(", ")
    println("  ${t.name}: $subjectNames" + if (t.unavailableDays.isNotEmpty()) " (off ${t.unavailableDays})" else "")
}


Parsed 10 teachers
  Dr. Smith: Math
  Ms. Johnson: Eng, Hist
  Prof. Chen: Chem
  Dr. Williams: Phys
  Ms. Brown: Bio
  Mr. Davis: CS
  Ms. Garcia: Art
  Mr. Miller: Mus
  Coach Wilson: PE
  Mrs. Taylor: Math, Eng (off [FRIDAY])


In [8]:
// ═══════════════════════════════════════════════════════════════════════════
// PARSE STUDENT GROUPS
// ═══════════════════════════════════════════════════════════════════════════

val studentGroups = schoolData.studentGroups.map { sg ->
    StudentGroup(
        id = sg.id,
        name = sg.name,
        size = sg.size,
        grade = sg.grade,
        hasWheelchairStudent = sg.hasWheelchairStudent
    )
}

val groupById = studentGroups.associateBy { it.id }

println("Parsed ${studentGroups.size} student groups")
studentGroups.filter { it.hasWheelchairStudent }.forEach {
    println("  ${it.name} requires wheelchair-accessible rooms")
}


Parsed 3 student groups


In [9]:
// ═══════════════════════════════════════════════════════════════════════════
// GENERATE LESSONS FROM CURRICULUM
// ═══════════════════════════════════════════════════════════════════════════

var lessonId = 0

// Curriculum is already parsed as Map<String, Int> by Jackson
val curriculum = schoolData.curriculum

// Find a teacher for a subject
fun findTeacher(subjectId: String): Teacher {
    return teachers.find { subjectId in it.subjects } 
        ?: throw IllegalStateException("No teacher for subject $subjectId")
}

val lessons = studentGroups.flatMap { group ->
    curriculum.entries.flatMap { (subjectId, count) ->
        val subject = subjectById[subjectId]!!
        val teacher = findTeacher(subjectId)
        (1..count).map { Lesson(lessonId++, subject, teacher, group) }
    }
}

val lessonById = lessons.associateBy { it.id }
val lessonsByTeacher = lessons.groupBy { it.teacher.id }
val lessonsByGroup = lessons.groupBy { it.studentGroup.id }

println("Generated ${lessons.size} lessons from curriculum")
println("  Per subject: ${lessons.groupBy { it.subject.shortName }.map { "${it.key}:${it.value.size}" }.joinToString(", ")}")


Generated 36 lessons from curriculum
  Per subject: Math:9, Eng:6, Chem:3, Phys:3, Bio:3, CS:3, Art:3, Mus:3, PE:3


In [10]:
// ═══════════════════════════════════════════════════════════════════════════
// PRECOMPUTE - Helper data structures for constraints
// ═══════════════════════════════════════════════════════════════════════════

val timeslotById = timeslots.associateBy { it.id }
val roomById = rooms.associateBy { it.id }
val timeslotsByDay = timeslots.groupBy { it.dayOfWeek }

// Find suitable rooms for each lesson (capacity + required capabilities)
val suitableRoomsForLesson = lessons.associate { lesson ->
    val suitable = rooms.filter { room ->
        room.capacity >= lesson.studentGroup.size &&
        room.hasAllCapabilities(lesson.subject.requiredCapabilities)
    }
    lesson.id to suitable.map { it.id }
}

// PE special case: must be in gym or outdoor
val peRooms = rooms.filter { 
    it.hasCapability(RoomCapability.GYM_EQUIPMENT) || it.hasCapability(RoomCapability.OUTDOOR_ACCESS)
}.map { it.id }

// Available timeslots per teacher (excluding unavailable days)
val availableTimeslotsForTeacher = teachers.associate { teacher ->
    teacher.id to timeslots.filter { it.dayOfWeek !in teacher.unavailableDays }.map { it.id }
}

// Consecutive timeslot pairs within each day
data class TimeslotPair(val first: Int, val second: Int, val day: DayOfWeek)
val consecutiveTimeslotPairs = timeslotsByDay.entries.flatMap { (day, slots) ->
    slots.sortedBy { it.periodNumber }.zipWithNext().map { (t1, t2) ->
        TimeslotPair(t1.id, t2.id, day)
    }
}

// Maximum allowed walking time (hard constraint)
val MAX_WALKING_TIME = 8  // minutes

println("Precomputed constraint helpers")
println("  Lessons with limited room options: ${suitableRoomsForLesson.count { it.value.size <= 2 }}")
println("  PE rooms available: ${peRooms.size}")


Precomputed constraint helpers
  Lessons with limited room options: 18
  PE rooms available: 3


## Building the Constraint Model

Define decision variables, hard constraints, soft constraints, and optimization objectives.


In [11]:
// ═══════════════════════════════════════════════════════════════════════════
// BUILD THE CONSTRAINT MODEL
// ═══════════════════════════════════════════════════════════════════════════

val numTimeslots = timeslots.size
val numRooms = rooms.size

val model = ConstraintSolverBuilder()
    .timeLimit(120)  // 2 minutes
    .numWorkers(8)
    .logProgress(true)
    
    // ───────────────────────────────────────────────────────────────────────
    // DECISION VARIABLES
    // ───────────────────────────────────────────────────────────────────────
    .variables { scope ->
        val vars = mutableMapOf<String, Expr>()
        lessons.forEach { lesson ->
            vars["slot_${lesson.id}"] = scope.int("slot_${lesson.id}", 0, (numTimeslots - 1).toLong())
            vars["room_${lesson.id}"] = scope.int("room_${lesson.id}", 0, (numRooms - 1).toLong())
        }
        vars
    }
    
    // ───────────────────────────────────────────────────────────────────────
    // HARD CONSTRAINT 1: Room Conflict (using allDifferent for efficiency)
    // No two lessons in the same room at the same time
    // Encoding: combined = slot * numRooms + room (unique per slot+room combo)
    // ───────────────────────────────────────────────────────────────────────
    .hard("room_conflict") { _, vars ->
        // Create combined slot+room identifier for each lesson
        val combinedVars = lessons.map { lesson ->
            val slot = vars["slot_${lesson.id}"]!!
            val room = vars["room_${lesson.id}"]!!
            // combined = slot * numRooms + room (gives unique value per timeslot+room)
            slot * numRooms.toLong() + room
        }
        // All must be different = no two lessons share same slot+room
        allDifferent(combinedVars)
    }
    
    // ───────────────────────────────────────────────────────────────────────
    // HARD CONSTRAINT 2: Teacher Conflict (using allDifferent)
    // A teacher can only teach one lesson at a time
    // ───────────────────────────────────────────────────────────────────────
    .hard("teacher_conflict") { _, vars ->
        forAll(lessonsByTeacher.values.filter { it.size > 1 }) { teacherLessons ->
            // All timeslots for this teacher's lessons must be different
            val slots = teacherLessons.map { vars["slot_${it.id}"]!! }
            allDifferent(slots)
        }
    }
    
    // ───────────────────────────────────────────────────────────────────────
    // HARD CONSTRAINT 3: Student Group Conflict (using allDifferent)
    // A student group can only attend one lesson at a time
    // ───────────────────────────────────────────────────────────────────────
    .hard("student_group_conflict") { _, vars ->
        forAll(lessonsByGroup.values.filter { it.size > 1 }) { groupLessons ->
            // All timeslots for this group's lessons must be different
            val slots = groupLessons.map { vars["slot_${it.id}"]!! }
            allDifferent(slots)
        }
    }
    
    // ───────────────────────────────────────────────────────────────────────
    // HARD CONSTRAINT 4: Room Capability Requirements
    // Rooms must have required equipment for the subject
    // ───────────────────────────────────────────────────────────────────────
    .hard("room_capabilities") { _, vars ->
        forAll(lessons) { lesson ->
            val suitable = suitableRoomsForLesson[lesson.id] ?: emptyList()
            if (suitable.isNotEmpty()) {
                vars["room_${lesson.id}"]!! inDomain suitable.map { it.toLong() }.toLongArray()
            } else {
                expr(1L) eq 1L  // Always true if no restrictions
            }
        }
    }
    
    // ───────────────────────────────────────────────────────────────────────
    // HARD CONSTRAINT 5: Teacher Availability
    // Teachers cannot teach on their unavailable days
    // ───────────────────────────────────────────────────────────────────────
    .hard("teacher_availability") { _, vars ->
        forAll(lessons.filter { it.teacher.unavailableDays.isNotEmpty() }) { lesson ->
            val available = availableTimeslotsForTeacher[lesson.teacher.id] ?: emptyList()
            vars["slot_${lesson.id}"]!! inDomain available.map { it.toLong() }.toLongArray()
        }
    }

println("Hard constraints defined (5 types)")


Hard constraints defined (5 types)


In [12]:
// Continue building the model with soft constraints
// SIMPLIFIED: Using only SAT-friendly linear constraints (no abs/iif chains)
val modelWithSoftConstraints = model
    // ───────────────────────────────────────────────────────────────────────
    // SOFT CONSTRAINT 1: Teacher Time Preference
    // Teachers prefer morning or afternoon slots (pure domain check)
    // ───────────────────────────────────────────────────────────────────────
    .soft("teacher_time_preference", weight = 5, priority = 1) { _, vars ->
        val morningSlots = timeslots.filter { it.isMorning }.map { it.id.toLong() }.toLongArray()
        val afternoonSlots = timeslots.filter { it.isAfternoon }.map { it.id.toLong() }.toLongArray()
        
        sum(lessons) { lesson ->
            val slot = vars["slot_${lesson.id}"]!!
            if (lesson.teacher.prefersMorning) {
                iif(slot inDomain afternoonSlots, 1L, 0L)
            } else {
                iif(slot inDomain morningSlots, 1L, 0L)
            }
        }
    }
    
    // ───────────────────────────────────────────────────────────────────────
    // SOFT CONSTRAINT 2: Prefer Earlier Slots (pure domain check)
    // ───────────────────────────────────────────────────────────────────────
    .soft("prefer_earlier", weight = 2, priority = 1) { _, vars ->
        val laterSlots = timeslots.filter { it.periodNumber >= 5 }.map { it.id.toLong() }.toLongArray()
        sum(lessons) { lesson ->
            iif(vars["slot_${lesson.id}"]!! inDomain laterSlots, 1L, 0L)
        }
    }
    
    // ───────────────────────────────────────────────────────────────────────
    // SOFT CONSTRAINT 3: Difficult Subjects in Morning (pure domain check)
    // ───────────────────────────────────────────────────────────────────────
    .soft("difficult_morning", weight = 4, priority = 1) { _, vars ->
        val afternoonSlots = timeslots.filter { it.isAfternoon }.map { it.id.toLong() }.toLongArray()
        sum(lessons.filter { it.subject.difficulty >= 4 }) { lesson ->
            iif(vars["slot_${lesson.id}"]!! inDomain afternoonSlots, 1L, 0L)
        }
    }
    
    // ───────────────────────────────────────────────────────────────────────
    // SOFT CONSTRAINT 4: Spread Lessons Across Week (SAT-friendly)
    // For same-subject lessons, penalize if on same day using domain checks
    // ───────────────────────────────────────────────────────────────────────
    .soft("lesson_spread", weight = 6, priority = 1) { _, vars ->
        val weekdays = listOf(DayOfWeek.MONDAY, DayOfWeek.TUESDAY, DayOfWeek.WEDNESDAY, DayOfWeek.THURSDAY, DayOfWeek.FRIDAY)
        
        sum(studentGroups) { group ->
            val groupLessons = lessonsByGroup[group.id] ?: emptyList()
            val bySubject = groupLessons.groupBy { it.subject.id }.filter { it.value.size >= 2 }
            
            sum(bySubject.values.toList()) { subjectLessons ->
                val pairs = subjectLessons.flatMapIndexed { i, l1 ->
                    subjectLessons.drop(i + 1).map { l2 -> l1 to l2 }
                }
                // For each day, check if both lessons are on that day
                sum(weekdays) { day ->
                    val daySlots = (timeslotsByDay[day] ?: emptyList()).map { it.id.toLong() }.toLongArray()
                    if (daySlots.isEmpty()) expr(0L)
                    else {
                        sum(pairs) { (l1, l2) ->
                            val l1OnDay: Expr = vars["slot_${l1.id}"]!! inDomain daySlots
                            val l2OnDay: Expr = vars["slot_${l2.id}"]!! inDomain daySlots
                            iif(l1OnDay and l2OnDay, 1L, 0L)
                        }
                    }
                }
            }
        }
    }

println("Soft constraints defined (4 SAT-friendly types)")


Soft constraints defined (4 SAT-friendly types)


In [13]:
// Continue with more soft constraints and objectives
// All constraints use SAT-friendly domain checks only
val completeModel = modelWithSoftConstraints
    // ───────────────────────────────────────────────────────────────────────
    // SOFT CONSTRAINT 5: Wheelchair Accessibility Priority (domain check)
    // ───────────────────────────────────────────────────────────────────────
    .soft("wheelchair_accessibility", weight = 15, priority = 1) { _, vars ->
        val accessibleRoomIds = rooms.filter { 
            it.hasCapability(RoomCapability.WHEELCHAIR_ACCESSIBLE) 
        }.map { it.id.toLong() }.toLongArray()
        
        sum(lessons.filter { it.studentGroup.hasWheelchairStudent }) { lesson ->
            val inAccessibleRoom: Expr = vars["room_${lesson.id}"]!! inDomain accessibleRoomIds
            iif(inAccessibleRoom.not(), 1L, 0L)
        }
    }
    
    // ═══════════════════════════════════════════════════════════════════════
    // MAXIMIZATION OBJECTIVE: Teacher Satisfaction Score (domain checks only)
    // ═══════════════════════════════════════════════════════════════════════
    .maximize("teacher_satisfaction") { _, vars ->
        val morningSlots = timeslots.filter { it.isMorning }.map { it.id.toLong() }.toLongArray()
        val afternoonSlots = timeslots.filter { it.isAfternoon }.map { it.id.toLong() }.toLongArray()
        
        sum(lessons) { lesson ->
            val slot = vars["slot_${lesson.id}"]!!
            val room = vars["room_${lesson.id}"]!!
            
            // Points for time preference match
            val timeScore = if (lesson.teacher.prefersMorning) {
                iif(slot inDomain morningSlots, 3L, 0L)
            } else {
                iif(slot inDomain afternoonSlots, 3L, 0L)
            }
            
            // Points for preferred room capabilities
            val preferredRoomIds = rooms.filter { r ->
                lesson.subject.preferredCapabilities.isNotEmpty() &&
                r.hasAnyCapability(lesson.subject.preferredCapabilities)
            }.map { it.id.toLong() }.toLongArray()
            
            val roomScore = if (preferredRoomIds.isNotEmpty()) {
                iif(room inDomain preferredRoomIds, 2L, 0L)
            } else {
                expr(1L)  // No preference = 1 point
            }
            
            timeScore + roomScore
        }
    }
    .build()

println("Complete model built with:")
println("  • 5 hard constraints (using efficient allDifferent)")
println("  • 5 soft constraints (pure domain checks)")
println("  • 1 maximization objective")


Complete model built with:
  • 5 hard constraints (using efficient allDifferent)
  • 5 soft constraints (pure domain checks)
  • 1 maximization objective


## Solving the Timetable

Run the constraint solver to find an optimal schedule.


In [14]:
// ═══════════════════════════════════════════════════════════════════════════
// PRE-SOLVE DIAGNOSTICS
// ═══════════════════════════════════════════════════════════════════════════

println("PRE-SOLVE DIAGNOSTICS")
println("═".repeat(60))
println()

// Model size estimates
println("Model Size Estimates:")
val numDecisionVars = lessons.size * 2  // slot + room per lesson
println("  Decision variables: $numDecisionVars (${lessons.size} lessons × 2)")

// Hard constraints now use efficient allDifferent encoding
println("  Hard constraints:")
println("    - Room conflict: 1 allDifferent on ${lessons.size} combined vars")
println("    - Teacher conflict: ${lessonsByTeacher.size} allDifferent calls")
println("    - Student conflict: ${lessonsByGroup.size} allDifferent calls")
println("    - Room capabilities: ${lessons.size} domain constraints")
println("    - Teacher availability: ${lessons.count { it.teacher.unavailableDays.isNotEmpty() }} domain constraints")

// Soft constraints now use pure domain checks
val softExprEstimate = lessons.size * 3 + // per-lesson domain checks
    lessons.count { it.studentGroup.hasWheelchairStudent } + // wheelchair
    lessonsByGroup.values.sumOf { grp ->
        val bySubj = grp.groupBy { it.subject.id }.filter { it.value.size >= 2 }
        bySubj.values.sumOf { it.size * (it.size - 1) / 2 * 5 } // pairs × days
    }
println("  Estimated soft expressions: ~$softExprEstimate (all linear domain checks)")
println()

// Check room capacity constraints
println("Room Capacity Analysis:")
val roomsWithCapacity = rooms.sortedByDescending { it.capacity }
roomsWithCapacity.take(5).forEach { r ->
    println("  ${r.name}: capacity ${r.capacity}, capabilities: ${r.capabilityCodes}")
}

// Check for lessons with limited room options
println("\nLessons with Limited Room Options:")
val limitedRoomLessons = suitableRoomsForLesson.filter { it.value.size <= 2 }
println("  ${limitedRoomLessons.size} lessons have ≤2 suitable rooms")
val noRoomLessons = suitableRoomsForLesson.filter { it.value.isEmpty() }
if (noRoomLessons.isNotEmpty()) {
    println("  ⚠️ WARNING: ${noRoomLessons.size} lessons have NO suitable rooms!")
    noRoomLessons.keys.take(5).forEach { id ->
        val lesson = lessonById[id]!!
        println("    - ${lesson.subject.name} for ${lesson.studentGroup.name} (size: ${lesson.studentGroup.size})")
    }
}

// Check teacher workload
println("\nTeacher Workload:")
lessonsByTeacher.forEach { (teacherId, lessons) ->
    val teacher = teacherById[teacherId]!!
    val availableSlots = availableTimeslotsForTeacher[teacherId]?.size ?: 0
    val status = if (lessons.size > availableSlots) "⚠️ OVERLOAD" else "OK"
    println("  ${teacher.name}: ${lessons.size} lessons, $availableSlots available slots - $status")
}

// Check student group workload
println("\nStudent Group Workload:")
lessonsByGroup.forEach { (groupId, lessons) ->
    val group = groupById[groupId]!!
    val status = if (lessons.size > timeslots.size) "⚠️ OVERLOAD" else "OK"
    println("  ${group.name}: ${lessons.size} lessons, ${timeslots.size} available slots - $status")
}

println()
println("═".repeat(60))
println()

// ═══════════════════════════════════════════════════════════════════════════
// SOLVE THE MODEL
// ═══════════════════════════════════════════════════════════════════════════

println("Solving timetabling model...")
println("   ${lessons.size} lessons x ${timeslots.size} timeslots x ${rooms.size} rooms")
println()

val startTime = System.currentTimeMillis()
val result = completeModel.solve()
val solveTime = (System.currentTimeMillis() - startTime) / 1000.0

println("Solve completed in ${"%.1f".format(solveTime)} seconds")
println()

when (result.status) {
    SolveStatus.OPTIMAL -> println("✓ OPTIMAL solution found!")
    SolveStatus.FEASIBLE -> println("✓ Feasible solution found (may not be optimal)")
    SolveStatus.INFEASIBLE -> println("✗ INFEASIBLE - constraints cannot be satisfied")
    SolveStatus.UNKNOWN -> println("? UNKNOWN - solver couldn't determine (try increasing time limit)")
    else -> println("Status: ${result.status}")
}

if (result.hasSolution) {
    println("\nSolution Statistics:")
    println("   Objective value (teacher satisfaction): ${result.objectiveValue}")
} else {
    println("\nNo solution found. Check the diagnostics above for potential issues.")
}


PRE-SOLVE DIAGNOSTICS
════════════════════════════════════════════════════════════

Model Size Estimates:
  Decision variables: 72 (36 lessons × 2)
  Hard constraints:
    - Room conflict: 1 allDifferent on 36 combined vars
    - Teacher conflict: 9 allDifferent calls
    - Student conflict: 3 allDifferent calls
    - Room capabilities: 36 domain constraints
    - Teacher availability: 0 domain constraints
  Estimated soft expressions: ~168 (all linear domain checks)

Room Capacity Analysis:
  Sports Field: capacity 100, capabilities: OUT
  Gymnasium: capacity 60, capabilities: GYM ACC
  Main 103: capacity 32, capabilities: PROJ WB
  Main 101: capacity 30, capabilities: PROJ WB ACC
  Main 202: capacity 30, capabilities: SB

Lessons with Limited Room Options:
  18 lessons have ≤2 suitable rooms

Teacher Workload:
  Dr. Smith: 9 lessons, 30 available slots - OK
  Ms. Johnson: 6 lessons, 30 available slots - OK
  Prof. Chen: 3 lessons, 30 available slots - OK
  Dr. Williams: 3 lessons, 30

In [15]:
// ═══════════════════════════════════════════════════════════════════════════
// EXTRACT SOLUTION
// ═══════════════════════════════════════════════════════════════════════════

data class ScheduledLesson(
    val lesson: Lesson,
    val timeslot: Timeslot,
    val room: Room
)

val scheduledLessons = if (result.hasSolution) {
    lessons.map { lesson ->
        val slotId = result.scope.value(result.variables["slot_${lesson.id}"]!!).getOrNull()!!.toInt()
        val roomId = result.scope.value(result.variables["room_${lesson.id}"]!!).getOrNull()!!.toInt()
        ScheduledLesson(
            lesson = lesson,
            timeslot = timeslotById[slotId]!!,
            room = roomById[roomId]!!
        )
    }
} else {
    emptyList()
}

if (scheduledLessons.isNotEmpty()) {
    println("Extracted ${scheduledLessons.size} scheduled lessons")
    
    // Quick summary
    val byDay = scheduledLessons.groupBy { it.timeslot.dayOfWeek }
    println("\nLessons per day: ${byDay.map { "${it.key.toString().take(3)}:${it.value.size}" }.joinToString(", ")}")
}


Extracted 36 scheduled lessons

Lessons per day: FRI:5, WED:9, THU:9, MON:6, TUE:7


## Visualizations

Modern, card-based timetable displays with walking distance indicators and room capability icons.


In [16]:
// ═══════════════════════════════════════════════════════════════════════════
// VISUALIZATION: Student Group Timetables
// ═══════════════════════════════════════════════════════════════════════════

if (scheduledLessons.isNotEmpty()) {
    val days = listOf(DayOfWeek.MONDAY, DayOfWeek.TUESDAY, DayOfWeek.WEDNESDAY, DayOfWeek.THURSDAY, DayOfWeek.FRIDAY)
    val periods = (1..6).toList()
    
    val css = """
        <style>
        .timetable-container {
            font-family: 'Segoe UI', system-ui, sans-serif;
            margin: 20px 0;
        }
        .timetable-title {
            font-size: 1.4em;
            font-weight: 600;
            margin-bottom: 15px;
            color: #1a1a2e;
            display: flex;
            align-items: center;
            gap: 10px;
        }
        .timetable-title .badge {
            font-size: 0.7em;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 4px 12px;
            border-radius: 20px;
            font-weight: 500;
        }
        .timetable {
            border-collapse: separate;
            border-spacing: 4px;
            width: 100%;
            background: #f8f9fa;
            border-radius: 12px;
            padding: 8px;
        }
        .timetable th {
            background: linear-gradient(135deg, #2d3436 0%, #636e72 100%);
            color: white;
            padding: 12px 8px;
            text-align: center;
            font-weight: 500;
            font-size: 0.85em;
            border-radius: 8px;
        }
        .timetable td {
            padding: 0;
            vertical-align: top;
            min-width: 140px;
            height: 90px;
        }
        .lesson-card {
            height: 100%;
            border-radius: 10px;
            padding: 10px;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
            transition: transform 0.2s, box-shadow 0.2s;
            cursor: default;
        }
        .lesson-card:hover {
            transform: translateY(-2px);
            box-shadow: 0 4px 12px rgba(0,0,0,0.15);
        }
        .lesson-subject {
            font-weight: 600;
            font-size: 0.95em;
            color: rgba(0,0,0,0.8);
        }
        .lesson-teacher {
            font-size: 0.75em;
            color: rgba(0,0,0,0.6);
            margin-top: 4px;
        }
        .lesson-room {
            font-size: 0.75em;
            color: rgba(0,0,0,0.7);
            background: rgba(255,255,255,0.5);
            padding: 3px 8px;
            border-radius: 12px;
            display: inline-flex;
            align-items: center;
            gap: 4px;
            margin-top: auto;
        }
        .lesson-room .icons {
            font-size: 0.9em;
        }
        .empty-slot {
            background: repeating-linear-gradient(
                45deg,
                #f0f0f0,
                #f0f0f0 10px,
                #f8f8f8 10px,
                #f8f8f8 20px
            );
            border-radius: 10px;
            height: 100%;
        }
        .period-header {
            font-size: 0.8em;
            color: #666;
        }
        .walking-indicator {
            position: absolute;
            right: -12px;
            top: 50%;
            transform: translateY(-50%);
            background: #ff6b6b;
            color: white;
            font-size: 0.65em;
            padding: 2px 6px;
            border-radius: 10px;
            z-index: 10;
        }
        .walking-indicator.short { background: #51cf66; }
        .walking-indicator.medium { background: #fcc419; color: #333; }
        .walking-indicator.long { background: #ff6b6b; }
        </style>
    """
    
    // Generate timetable for each student group
    val html = StringBuilder()
    html.append(css)
    
    studentGroups.forEach { group ->
        val groupLessons = scheduledLessons.filter { it.lesson.studentGroup.id == group.id }
        val bySlot = groupLessons.associateBy { it.timeslot.id }
        
        html.append("""
            <div class="timetable-container">
                <div class="timetable-title">
                    ${group.name}
                    <span class="badge">${group.size} students</span>
                    ${if (group.hasWheelchairStudent) "<span class=\"badge\">Accessible</span>" else ""}
                </div>
                <table class="timetable">
                    <tr>
                        <th></th>
                        ${days.joinToString("") { "<th>${it.toString().take(3)}</th>" }}
                    </tr>
        """)
        
        periods.forEach { period ->
            val periodSlots = timeslots.filter { it.periodNumber == period }
            val timeLabel = periodSlots.firstOrNull()?.let { "${it.startTime}" } ?: ""
            
            html.append("<tr><th><div>P$period</div><div class=\"period-header\">$timeLabel</div></th>")
            
            days.forEach { day ->
                val slot = periodSlots.find { it.dayOfWeek == day }
                val lesson = slot?.let { bySlot[it.id] }
                
                if (lesson != null) {
                    val bgColor = lesson.lesson.subject.color
                    html.append("""
                        <td>
                            <div class="lesson-card" style="background: $bgColor;">
                                <div>
                                    <div class="lesson-subject">${lesson.lesson.subject.name}</div>
                                    <div class="lesson-teacher">${lesson.lesson.teacher.name}</div>
                                </div>
                                <div class="lesson-room">
                                    <span class="icons">${lesson.room.capabilityCodes.take(12)}</span>
                                    ${lesson.room.name}
                                </div>
                            </div>
                        </td>
                    """)
                } else {
                    html.append("<td><div class=\"empty-slot\"></div></td>")
                }
            }
            html.append("</tr>")
        }
        
        html.append("</table></div>")
    }
    
    DISPLAY(HTML(html.toString()))
}


,MON,TUE,WED,THU,FRI
P108:00,English Literature Ms. Johnson PROJ WB ACC Main 101,,Chemistry Prof. Chen CHEM PROJ AC Chem Lab,Computer Science Mr. Davis COMP PROJ AC Computer Lab,Mathematics Dr. Smith PHYS PROJ SB Physics Lab
P209:00,,,Mathematics Dr. Smith PHYS PROJ SB Physics Lab,Mathematics Dr. Smith SB ACC Main 102,
P310:10,,,Biology Ms. Brown BIO PROJ ACC Bio Lab,,
P411:10,,English Literature Ms. Johnson COMP PROJ AC Computer Lab,,,Physics Dr. Williams PHYS PROJ SB Physics Lab
P513:00,Art Ms. Garcia ART OUT ACC Art Studio,,Music Mr. Miller MUS AUD ACC Music Room,Physical Education Coach Wilson OUT Sports Field,
P614:00,,,,,
,MON,TUE,WED,THU,FRI
P108:00,Physics Dr. Williams PHYS PROJ SB Physics Lab,Computer Science Mr. Davis COMP PROJ AC Computer Lab,,Biology Ms. Brown BIO PROJ ACC Bio Lab,
P209:00,,Mathematics Dr. Smith PHYS PROJ SB Physics Lab,English Literature Ms. Johnson COMP PROJ AC Computer Lab,,English Literature Ms. Johnson CHEM PROJ AC Chem Lab
P310:10,,,Chemistry Prof. Chen CHEM PROJ AC Chem Lab,,


In [17]:
// ═══════════════════════════════════════════════════════════════════════════
// VISUALIZATION: Teacher Workload Heatmap
// ═══════════════════════════════════════════════════════════════════════════

if (scheduledLessons.isNotEmpty()) {
    val days = listOf(DayOfWeek.MONDAY, DayOfWeek.TUESDAY, DayOfWeek.WEDNESDAY, DayOfWeek.THURSDAY, DayOfWeek.FRIDAY)
    
    val html = StringBuilder()
    html.append("""
        <style>
        .teacher-heatmap {
            font-family: 'Segoe UI', system-ui, sans-serif;
            margin: 30px 0;
        }
        .teacher-heatmap h2 {
            font-size: 1.4em;
            color: #1a1a2e;
            margin-bottom: 20px;
        }
        .teacher-grid {
            display: grid;
            grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
            gap: 16px;
        }
        .teacher-card {
            background: white;
            border-radius: 12px;
            padding: 16px;
            box-shadow: 0 2px 12px rgba(0,0,0,0.08);
            border: 1px solid #eee;
        }
        .teacher-name {
            font-weight: 600;
            font-size: 1.1em;
            color: #2d3436;
            margin-bottom: 12px;
            display: flex;
            align-items: center;
            gap: 8px;
        }
        .teacher-name .subjects {
            font-size: 0.7em;
            font-weight: 400;
            color: #636e72;
        }
        .day-bars {
            display: flex;
            gap: 6px;
            margin-bottom: 12px;
        }
        .day-bar {
            flex: 1;
            text-align: center;
        }
        .day-bar-label {
            font-size: 0.7em;
            color: #636e72;
            margin-bottom: 4px;
        }
        .day-bar-value {
            height: 40px;
            border-radius: 6px;
            display: flex;
            align-items: center;
            justify-content: center;
            font-weight: 600;
            color: white;
            font-size: 0.9em;
        }
        .bar-0 { background: #dfe6e9; color: #636e72; }
        .bar-1 { background: linear-gradient(135deg, #74b9ff, #0984e3); }
        .bar-2 { background: linear-gradient(135deg, #81ecec, #00b894); }
        .bar-3 { background: linear-gradient(135deg, #ffeaa7, #fdcb6e); color: #333; }
        .bar-4 { background: linear-gradient(135deg, #fab1a0, #e17055); }
        .bar-5, .bar-6 { background: linear-gradient(135deg, #fd79a8, #e84393); }
        .teacher-stats {
            display: flex;
            gap: 16px;
            font-size: 0.8em;
        }
        .stat {
            display: flex;
            align-items: center;
            gap: 4px;
        }
        .stat-icon { font-size: 1.2em; }
        .preference-match {
            display: inline-block;
            width: 8px;
            height: 8px;
            border-radius: 50%;
            margin-right: 4px;
        }
        .pref-good { background: #00b894; }
        .pref-bad { background: #d63031; }
        </style>
        <div class="teacher-heatmap">
            <h2>Teacher Workload Dashboard</h2>
            <div class="teacher-grid">
    """)
    
    teachers.forEach { teacher ->
        val teacherLessons = scheduledLessons.filter { it.lesson.teacher.id == teacher.id }
        val byDay = teacherLessons.groupBy { it.timeslot.dayOfWeek }
        val subjectNames = teacher.subjects.mapNotNull { subjectById[it]?.shortName }.joinToString(", ")
        
        // Calculate stats
        val totalLessons = teacherLessons.size
        val morningLessons = teacherLessons.count { it.timeslot.isMorning }
        val prefMatch = if (teacher.prefersMorning) morningLessons else (totalLessons - morningLessons)
        val prefPercent = if (totalLessons > 0) (prefMatch * 100) / totalLessons else 0
        
        // Calculate total walking distance
        var totalWalking = 0
        days.forEach { day ->
            val dayLessons = byDay[day]?.sortedBy { it.timeslot.periodNumber } ?: emptyList()
            dayLessons.zipWithNext().forEach { (l1, l2) ->
                if (l2.timeslot.periodNumber == l1.timeslot.periodNumber + 1) {
                    totalWalking += distanceMatrix[l1.room.id to l2.room.id] ?: 0
                }
            }
        }
        
        html.append("""
            <div class="teacher-card">
                <div class="teacher-name">
                    ${teacher.name}
                    <span class="subjects">($subjectNames)</span>
                </div>
                <div class="day-bars">
        """)
        
        days.forEach { day ->
            val count = byDay[day]?.size ?: 0
            html.append("""
                <div class="day-bar">
                    <div class="day-bar-label">${day.toString().take(3)}</div>
                    <div class="day-bar-value bar-$count">${if (count > 0) count else "-"}</div>
                </div>
            """)
        }
        
        val prefClass = if (prefPercent >= 60) "pref-good" else "pref-bad"
        html.append("""
                </div>
                <div class="teacher-stats">
                    <span class="stat">$totalLessons lessons</span>
                    <span class="stat"><span class="preference-match $prefClass"></span> $prefPercent% pref match</span>
                    <span class="stat">${totalWalking}min walking</span>
                </div>
            </div>
        """)
    }
    
    html.append("</div></div>")
    DISPLAY(HTML(html.toString()))
}


Teacher Workload Dashboard 
 
 
 
 
 Dr. Smith
 (Math) 
 
 
 
 
 MON 
 2 
 
 
 
 TUE 
 2 
 
 
 
 WED 
 3 
 
 
 
 THU 
 1 
 
 
 
 FRI 
 1 
 
 
 
 
 9 lessons 
 100% pref match 
 3min walking 
 
 
 
 
 
 Ms. Johnson
 (Eng, Hist) 
 
 
 
 
 MON 
 1 
 
 
 
 TUE 
 1 
 
 
 
 WED 
 1 
 
 
 
 THU 
 1 
 
 
 
 FRI 
 2 
 
 
 
 
 6 lessons 
 100% pref match 
 1min walking 
 
 
 
 
 
 Prof. Chen
 (Chem) 
 
 
 
 
 MON 
 1 
 
 
 
 TUE 
 - 
 
 
 
 WED 
 2 
 
 
 
 THU 
 - 
 
 
 
 FRI 
 - 
 
 
 
 
 3 lessons 
 100% pref match 
 0min walking 
 
 
 
 
 
 Dr. Williams
 (Phys) 
 
 
 
 
 MON 
 1 
 
 
 
 TUE 
 - 
 
 
 
 WED 
 - 
 
 
 
 THU 
 1 
 
 
 
 FRI 
 1 
 
 
 
 
 3 lessons 
 100% pref match 
 0min walking 
 
 
 
 
 
 Ms. Brown
 (Bio) 
 
 
 
 
 MON 
 - 
 
 
 
 TUE 
 - 
 
 
 
 WED 
 1 
 
 
 
 THU 
 2 
 
 
 
 FRI 
 - 
 
 
 
 
 3 lessons 
 100% pref match 
 0min walking 
 
 
 
 
 
 Mr. Davis
 (CS) 
 
 
 
 
 MON 
 - 
 
 
 
 TUE 
 1 
 
 
 
 WED 
 - 
 
 
 
 THU 
 1 
 
 
 
 FRI 
 1 
 
 
 
 
 3 lessons 
 0% pref match 
 0min walking 
 
 
 
 
 
 Ms. Garcia
 (Art) 
 
 
 
 
 MON 
 1 
 
 
 
 TUE 
 1 
 
 
 
 WED 
 1 
 
 
 
 THU 
 - 
 
 
 
 FRI 
 - 
 
 
 
 
 3 lessons 
 100% pref match 
 0min walking 
 
 
 
 
 
 Mr. Miller
 (Mus) 
 
 
 
 
 MON 
 - 
 
 
 
 TUE 
 1 
 
 
 
 WED 
 1 
 
 
 
 THU 
 1 
 
 
 
 FRI 
 - 
 
 
 
 
 3 lessons 
 100% pref match 
 0min walking 
 
 
 
 
 
 Coach Wilson
 (PE) 
 
 
 
 
 MON 
 - 
 
 
 
 TUE 
 1 
 
 
 
 WED 
 - 
 
 
 
 THU 
 2 
 
 
 
 FRI 
 - 
 
 
 
 
 3 lessons 
 100% pref match 
 1min walking 
 
 
 
 
 
 Mrs. Taylor
 (Math, Eng) 
 
 
 
 
 MON 
 - 
 
 
 
 TUE 
 - 
 
 
 
 WED 
 - 
 
 
 
 THU 
 - 
 
 
 
 FRI 
 - 
 
 
 
 
 0 lessons 
 0% pref match 
 0min walking

In [18]:
// ═══════════════════════════════════════════════════════════════════════════
// VISUALIZATION: Room Utilization Dashboard
// ═══════════════════════════════════════════════════════════════════════════

if (scheduledLessons.isNotEmpty()) {
    val totalSlots = timeslots.size
    
    val html = StringBuilder()
    html.append("""
        <style>
        .room-dashboard {
            font-family: 'Segoe UI', system-ui, sans-serif;
            margin: 30px 0;
        }
        .room-dashboard h2 {
            font-size: 1.4em;
            color: #1a1a2e;
            margin-bottom: 20px;
        }
        .room-grid {
            display: grid;
            grid-template-columns: repeat(auto-fill, minmax(320px, 1fr));
            gap: 12px;
        }
        .room-row {
            background: white;
            border-radius: 10px;
            padding: 14px 18px;
            display: flex;
            align-items: center;
            gap: 16px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.06);
            border: 1px solid #eee;
        }
        .room-info {
            min-width: 120px;
        }
        .room-name {
            font-weight: 600;
            color: #2d3436;
        }
        .room-building {
            font-size: 0.75em;
            color: #636e72;
        }
        .room-icons {
            font-size: 0.85em;
            margin-top: 4px;
        }
        .utilization-bar-container {
            flex: 1;
            display: flex;
            align-items: center;
            gap: 10px;
        }
        .utilization-bar {
            flex: 1;
            height: 24px;
            background: #f0f0f0;
            border-radius: 12px;
            overflow: hidden;
            position: relative;
        }
        .utilization-fill {
            height: 100%;
            border-radius: 12px;
            transition: width 0.3s;
        }
        .util-low { background: linear-gradient(90deg, #74b9ff, #0984e3); }
        .util-medium { background: linear-gradient(90deg, #ffeaa7, #f39c12); }
        .util-high { background: linear-gradient(90deg, #ff7675, #d63031); }
        .utilization-percent {
            min-width: 50px;
            font-weight: 600;
            font-size: 0.95em;
            text-align: right;
        }
        .room-capacity {
            font-size: 0.75em;
            color: #636e72;
            min-width: 60px;
            text-align: right;
        }
        .building-section {
            margin-bottom: 24px;
        }
        .building-header {
            font-size: 1.1em;
            font-weight: 600;
            color: #636e72;
            margin-bottom: 12px;
            padding-left: 4px;
            border-left: 4px solid #667eea;
            padding-left: 12px;
        }
        </style>
        <div class="room-dashboard">
            <h2>Room Utilization</h2>
    """)
    
    val byRoom = scheduledLessons.groupBy { it.room.id }
    val byBuilding = rooms.groupBy { it.building }
    
    byBuilding.forEach { (building, buildingRooms) ->
        html.append("""<div class="building-section"><div class="building-header">$building</div><div class="room-grid">""")
        
        buildingRooms.sortedBy { it.name }.forEach { room ->
            val lessonCount = byRoom[room.id]?.size ?: 0
            val utilPercent = (lessonCount * 100) / totalSlots
            val utilClass = when {
                utilPercent < 40 -> "util-low"
                utilPercent < 70 -> "util-medium"
                else -> "util-high"
            }
            
            html.append("""
                <div class="room-row">
                    <div class="room-info">
                        <div class="room-name">${room.name}</div>
                        <div class="room-building">Floor ${room.floor}</div>
                        <div class="room-icons">${room.capabilityCodes}</div>
                    </div>
                    <div class="utilization-bar-container">
                        <div class="utilization-bar">
                            <div class="utilization-fill $utilClass" style="width: $utilPercent%;"></div>
                        </div>
                        <div class="utilization-percent">$utilPercent%</div>
                    </div>
                    <div class="room-capacity">Capacity: ${room.capacity}</div>
                </div>
            """)
        }
        
        html.append("</div></div>")
    }
    
    html.append("</div>")
    DISPLAY(HTML(html.toString()))
}


Room Utilization 
 Main 
 
 
 Main 101 
 Floor 1 
 PROJ WB ACC 
 
 
 
 
 
 3% 
 
 Capacity: 30 
 
 
 
 
 Main 102 
 Floor 1 
 SB ACC 
 
 
 
 
 
 13% 
 
 Capacity: 28 
 
 
 
 
 Main 103 
 Floor 1 
 PROJ WB 
 
 
 
 
 
 0% 
 
 Capacity: 32 
 
 
 
 
 Main 201 
 Floor 2 
 PROJ WB 
 
 
 
 
 
 0% 
 
 Capacity: 28 
 
 
 
 
 Main 202 
 Floor 2 
 SB 
 
 
 
 
 
 3% 
 
 Capacity: 30 
 
 Science 
 
 
 Bio Lab 
 Floor 2 
 BIO PROJ ACC 
 
 
 
 
 
 10% 
 
 Capacity: 28 
 
 
 
 
 Chem Lab 
 Floor 1 
 CHEM PROJ ACC 
 
 
 
 
 
 13% 
 
 Capacity: 28 
 
 
 
 
 Computer Lab 
 Floor 2 
 COMP PROJ ACC 
 
 
 
 
 
 23% 
 
 Capacity: 30 
 
 
 
 
 Physics Lab 
 Floor 1 
 PHYS PROJ SB ACC 
 
 
 
 
 
 23% 
 
 Capacity: 28 
 
 Arts 
 
 
 Art Studio 
 Floor 1 
 ART OUT ACC 
 
 
 
 
 
 10% 
 
 Capacity: 28 
 
 
 
 
 Music Room 
 Floor 1 
 MUS AUD ACC 
 
 
 
 
 
 10% 
 
 Capacity: 28 
 
 Sports 
 
 
 Dance Studio 
 Floor 1 
 AUD ACC 
 
 
 
 
 
 0% 
 
 Capacity: 28 
 
 
 
 
 Gymnasium 
 Floor 0 
 GYM ACC 
 
 
 
 
 
 6% 
 
 Capacity: 60 
 
 
 
 
 Sports Field 
 Floor 0 
 OUT 
 
 
 
 
 
 3% 
 
 Capacity: 100

In [19]:
// ═══════════════════════════════════════════════════════════════════════════
// VISUALIZATION: Walking Distance Analysis
// ═══════════════════════════════════════════════════════════════════════════

if (scheduledLessons.isNotEmpty()) {
    val days = listOf(DayOfWeek.MONDAY, DayOfWeek.TUESDAY, DayOfWeek.WEDNESDAY, DayOfWeek.THURSDAY, DayOfWeek.FRIDAY)
    
    // Calculate walking stats for each student group
    data class WalkingStats(
        val group: StudentGroup,
        val totalMinutes: Int,
        val buildingTransitions: Int,
        val worstTransition: Triple<Room, Room, Int>?
    )
    
    val walkingStats = studentGroups.map { group ->
        val groupLessons = scheduledLessons.filter { it.lesson.studentGroup.id == group.id }
        var totalMinutes = 0
        var buildingTransitions = 0
        var worstTransition: Triple<Room, Room, Int>? = null
        
        days.forEach { day ->
            val dayLessons = groupLessons
                .filter { it.timeslot.dayOfWeek == day }
                .sortedBy { it.timeslot.periodNumber }
            
            dayLessons.zipWithNext().forEach { (l1, l2) ->
                // Only count if consecutive periods
                if (l2.timeslot.periodNumber == l1.timeslot.periodNumber + 1) {
                    val dist = distanceMatrix[l1.room.id to l2.room.id] ?: 0
                    totalMinutes += dist
                    
                    if (l1.room.building != l2.room.building) {
                        buildingTransitions++
                    }
                    
                    if (worstTransition == null || dist > worstTransition!!.third) {
                        worstTransition = Triple(l1.room, l2.room, dist)
                    }
                }
            }
        }
        
        WalkingStats(group, totalMinutes, buildingTransitions, worstTransition)
    }.sortedByDescending { it.totalMinutes }
    
    val html = StringBuilder()
    html.append("""
        <style>
        .walking-analysis {
            font-family: 'Segoe UI', system-ui, sans-serif;
            margin: 30px 0;
        }
        .walking-analysis h2 {
            font-size: 1.4em;
            color: #1a1a2e;
            margin-bottom: 20px;
        }
        .walking-table {
            width: 100%;
            border-collapse: separate;
            border-spacing: 0 8px;
        }
        .walking-table th {
            text-align: left;
            padding: 12px 16px;
            font-weight: 500;
            color: #636e72;
            font-size: 0.85em;
            border-bottom: 2px solid #dfe6e9;
        }
        .walking-table td {
            padding: 14px 16px;
            background: white;
            border: 1px solid #eee;
        }
        .walking-table tr td:first-child {
            border-radius: 10px 0 0 10px;
            border-right: none;
        }
        .walking-table tr td:last-child {
            border-radius: 0 10px 10px 0;
            border-left: none;
        }
        .walking-table tr td:not(:first-child):not(:last-child) {
            border-left: none;
            border-right: none;
        }
        .group-name {
            font-weight: 600;
            color: #2d3436;
        }
        .walking-bar {
            height: 20px;
            background: #f0f0f0;
            border-radius: 10px;
            overflow: hidden;
            min-width: 150px;
        }
        .walking-bar-fill {
            height: 100%;
            border-radius: 10px;
            background: linear-gradient(90deg, #00b894, #00cec9);
        }
        .walking-bar-fill.warning {
            background: linear-gradient(90deg, #fdcb6e, #e17055);
        }
        .walking-bar-fill.danger {
            background: linear-gradient(90deg, #e17055, #d63031);
        }
        .transition-badge {
            display: inline-block;
            padding: 4px 10px;
            border-radius: 12px;
            font-size: 0.85em;
            font-weight: 500;
        }
        .transition-badge.low { background: #d4edda; color: #155724; }
        .transition-badge.medium { background: #fff3cd; color: #856404; }
        .transition-badge.high { background: #f8d7da; color: #721c24; }
        .worst-route {
            font-size: 0.85em;
            color: #636e72;
        }
        .worst-route .arrow { color: #e17055; font-weight: bold; }
        </style>
        <div class="walking-analysis">
            <h2>Walking Distance Analysis</h2>
            <table class="walking-table">
                <tr>
                    <th>Student Group</th>
                    <th>Total Walking Time</th>
                    <th>Building Transitions</th>
                    <th>Longest Walk</th>
                </tr>
    """)
    
    val maxWalking = walkingStats.maxOfOrNull { it.totalMinutes } ?: 1
    
    walkingStats.forEach { stats ->
        val barPercent = (stats.totalMinutes * 100) / maxOf(maxWalking, 1)
        val barClass = when {
            stats.totalMinutes > 40 -> "danger"
            stats.totalMinutes > 25 -> "warning"
            else -> ""
        }
        val transitionClass = when {
            stats.buildingTransitions > 8 -> "high"
            stats.buildingTransitions > 4 -> "medium"
            else -> "low"
        }
        val worstText = stats.worstTransition?.let { (r1, r2, dist) ->
            "${r1.name} <span class='arrow'>→</span> ${r2.name} (${dist}min)"
        } ?: "-"
        
        html.append("""
            <tr>
                <td>
                    <span class="group-name">${stats.group.name}</span>
                    ${if (stats.group.hasWheelchairStudent) " [A]" else ""}
                </td>
                <td>
                    <div style="display: flex; align-items: center; gap: 12px;">
                        <div class="walking-bar">
                            <div class="walking-bar-fill $barClass" style="width: $barPercent%;"></div>
                        </div>
                        <span>${stats.totalMinutes} min</span>
                    </div>
                </td>
                <td><span class="transition-badge $transitionClass">${stats.buildingTransitions} changes</span></td>
                <td><span class="worst-route">$worstText</span></td>
            </tr>
        """)
    }
    
    html.append("</table></div>")
    DISPLAY(HTML(html.toString()))
}


Student Group,Total Walking Time,Building Transitions,Longest Walk
Grade 11A,8 min,1 changes,Main 102 → Art Studio (5min)
Grade 9A,6 min,1 changes,Computer Lab → Main 102 (4min)
Grade 10A,3 min,0 changes,Computer Lab → Physics Lab (1min)


## Solution Quality Analysis

Comprehensive metrics evaluating constraint satisfaction and optimization objectives.


In [20]:
// ═══════════════════════════════════════════════════════════════════════════
// SOLUTION QUALITY ANALYSIS
// ═══════════════════════════════════════════════════════════════════════════

if (scheduledLessons.isNotEmpty()) {
    println("╔══════════════════════════════════════════════════════════════════╗")
    println("║                    SOLUTION QUALITY REPORT                       ║")
    println("╚══════════════════════════════════════════════════════════════════╝")
    println()
    
    // ─────────────────────────────────────────────────────────────────────────
    // Hard Constraint Verification
    // ─────────────────────────────────────────────────────────────────────────
    println("HARD CONSTRAINT VERIFICATION")
    println("─".repeat(50))
    
    // Room conflicts
    val roomConflicts = scheduledLessons.groupBy { it.timeslot.id to it.room.id }
        .filter { it.value.size > 1 }
    println("  Room conflicts:        ${if (roomConflicts.isEmpty()) "None" else "${roomConflicts.size} VIOLATIONS"}")
    
    // Teacher conflicts  
    val teacherConflicts = scheduledLessons.groupBy { it.timeslot.id to it.lesson.teacher.id }
        .filter { it.value.size > 1 }
    println("  Teacher conflicts:     ${if (teacherConflicts.isEmpty()) "None" else "${teacherConflicts.size} VIOLATIONS"}")
    
    // Student group conflicts
    val studentConflicts = scheduledLessons.groupBy { it.timeslot.id to it.lesson.studentGroup.id }
        .filter { it.value.size > 1 }
    println("  Student conflicts:     ${if (studentConflicts.isEmpty()) "None" else "${studentConflicts.size} VIOLATIONS"}")
    
    // Capability violations
    val capabilityViolations = scheduledLessons.filter { sl ->
        !sl.room.hasAllCapabilities(sl.lesson.subject.requiredCapabilities)
    }
    println("  Capability violations: ${if (capabilityViolations.isEmpty()) "None" else "${capabilityViolations.size} VIOLATIONS"}")
    
    // Capacity violations
    val capacityViolations = scheduledLessons.filter { sl ->
        sl.room.capacity < sl.lesson.studentGroup.size
    }
    println("  Capacity violations:   ${if (capacityViolations.isEmpty()) "None" else "${capacityViolations.size} VIOLATIONS"}")
    
    println()
    
    // ─────────────────────────────────────────────────────────────────────────
    // Soft Constraint Metrics
    // ─────────────────────────────────────────────────────────────────────────
    println("SOFT CONSTRAINT METRICS")
    println("─".repeat(50))
    
    // Teacher time preference satisfaction
    val morningSlotIds = timeslots.filter { it.isMorning }.map { it.id }.toSet()
    val timePreferenceMatch = scheduledLessons.count { sl ->
        val inMorning = sl.timeslot.id in morningSlotIds
        (sl.lesson.teacher.prefersMorning && inMorning) || (!sl.lesson.teacher.prefersMorning && !inMorning)
    }
    val timePrefPercent = (timePreferenceMatch * 100) / lessons.size
    println("  Teacher time preference:    $timePrefPercent% satisfied")
    
    // Preferred room match
    val preferredRoomMatch = scheduledLessons.count { sl ->
        sl.lesson.subject.preferredCapabilities.isEmpty() ||
        sl.room.hasAnyCapability(sl.lesson.subject.preferredCapabilities)
    }
    val roomPrefPercent = (preferredRoomMatch * 100) / lessons.size
    println("  Preferred room match:       $roomPrefPercent% satisfied")
    
    // Difficult subjects in morning
    val difficultLessons = scheduledLessons.filter { it.lesson.subject.difficulty >= 4 }
    val difficultInMorning = difficultLessons.count { it.timeslot.isMorning }
    val diffMorningPercent = if (difficultLessons.isNotEmpty()) 
        (difficultInMorning * 100) / difficultLessons.size else 100
    println("  Difficult subjects AM:      $diffMorningPercent% (${difficultInMorning}/${difficultLessons.size})")
    
    // Wheelchair accessibility
    val wheelchairLessons = scheduledLessons.filter { it.lesson.studentGroup.hasWheelchairStudent }
    val wheelchairAccessible = wheelchairLessons.count { 
        it.room.hasCapability(RoomCapability.WHEELCHAIR_ACCESSIBLE) 
    }
    val wheelchairPercent = if (wheelchairLessons.isNotEmpty())
        (wheelchairAccessible * 100) / wheelchairLessons.size else 100
    println("  Wheelchair accessible:      $wheelchairPercent% (${wheelchairAccessible}/${wheelchairLessons.size})")
    
    println()
    
    // ─────────────────────────────────────────────────────────────────────────
    // Overall Statistics
    // ─────────────────────────────────────────────────────────────────────────
    println("OVERALL STATISTICS")
    println("─".repeat(50))
    
    val roomUtilization = rooms.map { room ->
        val lessonCount = scheduledLessons.count { it.room.id == room.id }
        room to lessonCount
    }
    val avgUtilization = roomUtilization.map { it.second }.average()
    val maxUtilization = roomUtilization.maxByOrNull { it.second }
    val minUtilization = roomUtilization.filter { it.second > 0 }.minByOrNull { it.second }
    
    println("  Total lessons scheduled:    ${scheduledLessons.size}")
    println("  Average room utilization:   ${"%.1f".format(avgUtilization)} lessons/room")
    println("  Most used room:             ${maxUtilization?.first?.name} (${maxUtilization?.second} lessons)")
    println("  Least used room:            ${minUtilization?.first?.name} (${minUtilization?.second} lessons)")
    
    val unusedRooms = roomUtilization.filter { it.second == 0 }.map { it.first.name }
    if (unusedRooms.isNotEmpty()) {
        println("  Unused rooms:               ${unusedRooms.joinToString(", ")}")
    }
    
    println()
    println("╔══════════════════════════════════════════════════════════════════╗")
    println("║  Final Objective Value (Teacher Satisfaction): ${result.objectiveValue}".padEnd(67) + "║")
    println("╚══════════════════════════════════════════════════════════════════╝")
}


╔══════════════════════════════════════════════════════════════════╗
║                    SOLUTION QUALITY REPORT                       ║
╚══════════════════════════════════════════════════════════════════╝

HARD CONSTRAINT VERIFICATION
──────────────────────────────────────────────────
  Room conflicts:        None
  Teacher conflicts:     None
  Student conflicts:     None
  Capability violations: None
  Capacity violations:   None

SOFT CONSTRAINT METRICS
──────────────────────────────────────────────────
  Teacher time preference:    91% satisfied
  Preferred room match:       100% satisfied
  Difficult subjects AM:      100% (21/21)
  Wheelchair accessible:      100% (0/0)

OVERALL STATISTICS
──────────────────────────────────────────────────
  Total lessons scheduled:    36
  Average room utilization:   2,6 lessons/room
  Most used room:             Physics Lab (7 lessons)
  Least used room:            Main 101 (1 lessons)
  Unused rooms:               Main 103, Main 201, Dance S